## Data Preprocessing for COVID 19 DSPG activity

In [1]:
%config InlineBackend.figure_format='retina'

import calendar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams, ticker
from scipy.optimize import curve_fit

from sklearn.neural_network import MLPClassifier

In [2]:
census_raw_url = "https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/counties/totals/co-est2019-alldata.csv"
census_raw_df  = pd.read_csv(census_raw_url, encoding="ISO-8859-1") 
census_raw_df.head()
census_county_df = census_raw_df[census_raw_df["SUMLEV"] == 50]
census_df = census_county_df[['STNAME', 'CTYNAME', 'STATE', 'COUNTY', 'POPESTIMATE2019']].copy()
census_df['COUNTYFP'] = ((census_df['STATE'] * 1000) + (census_df['COUNTY']))
census_df.head()

,STNAME,CTYNAME,STATE,COUNTY,POPESTIMATE2019,COUNTYFP
1,Alabama,Autauga County,1,1,55869,1001
2,Alabama,Baldwin County,1,3,223234,1003
3,Alabama,Barbour County,1,5,24686,1005
4,Alabama,Bibb County,1,7,22394,1007
5,Alabama,Blount County,1,9,57826,1009


In [3]:
mask_use_url = 'https://github.com/nytimes/covid-19-data/raw/master/mask-use/mask-use-by-county.csv'
mask_use_raw_df = pd.read_csv(mask_use_url, encoding="ISO-8859-1")
mask_use_raw_df.head()

,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
0,1001,0.053,0.074,0.134,0.295,0.444
1,1003,0.083,0.059,0.098,0.323,0.436
2,1005,0.067,0.121,0.120,0.201,0.491
3,1007,0.020,0.034,0.096,0.278,0.572
4,1009,0.053,0.114,0.180,0.194,0.459


In [4]:
int_table_01 = pd.merge(census_df, mask_use_raw_df, on="COUNTYFP", how="left")
int_table_01.head()

,STNAME,CTYNAME,STATE,COUNTY,POPESTIMATE2019,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
0,Alabama,Autauga County,1,1,55869,1001,0.053,0.074,0.134,0.295,0.444
1,Alabama,Baldwin County,1,3,223234,1003,0.083,0.059,0.098,0.323,0.436
2,Alabama,Barbour County,1,5,24686,1005,0.067,0.121,0.120,0.201,0.491
3,Alabama,Bibb County,1,7,22394,1007,0.020,0.034,0.096,0.278,0.572
4,Alabama,Blount County,1,9,57826,1009,0.053,0.114,0.180,0.194,0.459


In [5]:
county_covid_case_url = 'https://github.com/nytimes/covid-19-data/raw/master/us-counties.csv'
county_covid_case_raw_df = pd.read_csv(county_covid_case_url, encoding="ISO-8859-1", parse_dates=['date'])

In [6]:
gb_month_year = county_covid_case_raw_df.groupby(
    [county_covid_case_raw_df['date'].dt.year.rename('year'), 
     county_covid_case_raw_df['date'].dt.month.rename('month'),
     'fips', 'county', 'state', 
    ])[['deaths', 'cases']].agg('max').copy()
gb_month_year.reset_index(inplace=True)
gb_month_year['fips'] = gb_month_year['fips'].astype('int')

county_covid_2020_df = gb_month_year[gb_month_year["year"] == 2020].copy()

In [7]:
eoy_df = county_covid_2020_df[county_covid_2020_df["month"] == 12][['fips']].copy()
for i in np.arange(1, 12):

    # grab this month
    this_month = county_covid_2020_df[county_covid_2020_df["month"] == i][["fips", "cases", "deaths"]].copy()
    this_month.fillna(0, inplace=True)

    # not january
    if i > 1:
        # get the previous month
        previous_month = county_covid_2020_df[county_covid_2020_df["month"] == (i - 1)][["fips", "cases", "deaths"]].copy()
        previous_month = previous_month.rename(columns={"deaths": "deaths_prev", "cases": "cases_prev"})
        previous_month.fillna(0, inplace=True)

        # combine them to do math
        combined_df = pd.merge(this_month, previous_month[['fips', "deaths_prev", "cases_prev"]]) 
        
        combined_df[f"cases_{calendar.month_abbr[i].lower()}"] = combined_df['cases'] - combined_df['cases_prev']
        combined_df[f"deaths_{calendar.month_abbr[i].lower()}"] = combined_df['deaths'] - combined_df['deaths_prev']

    else:
        combined_df = this_month.copy()
        combined_df[f"cases_{calendar.month_abbr[i].lower()}"] = this_month['cases']
        combined_df[f"deaths_{calendar.month_abbr[i].lower()}"] = this_month['deaths']
        
        #print(combined_df.head())
    #this_month = this_month.rename(columns={"deaths": f"deaths_{calendar.month_abbr[i].lower()}", "cases": f"cases_{calendar.month_abbr[i].lower()}"}).copy()
    eoy_df = pd.merge(eoy_df, combined_df[['fips', f"cases_{calendar.month_abbr[i].lower()}", f"deaths_{calendar.month_abbr[i].lower()}"]], on="fips", how="left", ).copy()
eoy_df['COUNTYFP'] = eoy_df['fips'] 
eoy_df.head()

,fips,cases_jan,deaths_jan,cases_feb,deaths_feb,cases_mar,deaths_mar,cases_apr,deaths_apr,cases_may,deaths_may,cases_jun,deaths_jun,cases_jul,deaths_jul,cases_aug,deaths_aug,cases_sep,deaths_sep,cases_oct,deaths_oct,cases_nov,deaths_nov,COUNTYFP
0,1001,NaN,NaN,NaN,NaN,NaN,NaN,36.0,4.0,178.0,0.0,316.0,8.0,478.0,9.0,427.0,2.0,349.0,4.0,368.0,4.0,621.0,11.0,1001
1,1003,NaN,NaN,NaN,NaN,NaN,NaN,155.0,3.0,116.0,6.0,390.0,1.0,2421.0,12.0,1444.0,20.0,1095.0,10.0,1300.0,19.0,1950.0,27.0,1003
2,1005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,128.0,0.0,158.0,0.0,273.0,4.0,159.0,2.0,139.0,0.0,164.0,2.0,118.0,2.0,1005
3,1007,NaN,NaN,NaN,NaN,NaN,NaN,39.0,0.0,33.0,1.0,95.0,0.0,193.0,1.0,191.0,6.0,110.0,3.0,209.0,4.0,323.0,3.0,1007
4,1009,NaN,NaN,NaN,NaN,NaN,NaN,32.0,0.0,25.0,1.0,146.0,0.0,559.0,2.0,520.0,8.0,342.0,4.0,445.0,10.0,923.0,15.0,1009


In [8]:
eoy_df.head()
eoy_df.fillna(0, inplace=True)

In [9]:
export = pd.merge(int_table_01, eoy_df, on="COUNTYFP", how="inner", ).copy()
export.head()

,STNAME,CTYNAME,STATE,COUNTY,POPESTIMATE2019,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS,fips,cases_jan,deaths_jan,cases_feb,deaths_feb,cases_mar,deaths_mar,cases_apr,deaths_apr,cases_may,deaths_may,cases_jun,deaths_jun,cases_jul,deaths_jul,cases_aug,deaths_aug,cases_sep,deaths_sep,cases_oct,deaths_oct,cases_nov,deaths_nov
0,Alabama,Autauga County,1,1,55869,1001,0.053,0.074,0.134,0.295,0.444,1001,0.0,0.0,0.0,0.0,0.0,0.0,36.0,4.0,178.0,0.0,316.0,8.0,478.0,9.0,427.0,2.0,349.0,4.0,368.0,4.0,621.0,11.0
1,Alabama,Baldwin County,1,3,223234,1003,0.083,0.059,0.098,0.323,0.436,1003,0.0,0.0,0.0,0.0,0.0,0.0,155.0,3.0,116.0,6.0,390.0,1.0,2421.0,12.0,1444.0,20.0,1095.0,10.0,1300.0,19.0,1950.0,27.0
2,Alabama,Barbour County,1,5,24686,1005,0.067,0.121,0.120,0.201,0.491,1005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,128.0,0.0,158.0,0.0,273.0,4.0,159.0,2.0,139.0,0.0,164.0,2.0,118.0,2.0
3,Alabama,Bibb County,1,7,22394,1007,0.020,0.034,0.096,0.278,0.572,1007,0.0,0.0,0.0,0.0,0.0,0.0,39.0,0.0,33.0,1.0,95.0,0.0,193.0,1.0,191.0,6.0,110.0,3.0,209.0,4.0,323.0,3.0
4,Alabama,Blount County,1,9,57826,1009,0.053,0.114,0.180,0.194,0.459,1009,0.0,0.0,0.0,0.0,0.0,0.0,32.0,0.0,25.0,1.0,146.0,0.0,559.0,2.0,520.0,8.0,342.0,4.0,445.0,10.0,923.0,15.0


In [10]:
export_csv = export[[
       'STNAME', 'CTYNAME', 'COUNTYFP','STATE', 'POPESTIMATE2019', 
       'NEVER', 'RARELY', 'SOMETIMES', 'FREQUENTLY', 'ALWAYS','cases_jan',
       'deaths_jan', 'cases_feb', 'deaths_feb', 'cases_mar', 'deaths_mar',
       'cases_apr', 'deaths_apr', 'cases_may', 'deaths_may', 'cases_jun',
       'deaths_jun', 'cases_jul', 'deaths_jul', 'cases_aug', 'deaths_aug',
       'cases_sep', 'deaths_sep', 'cases_oct', 'deaths_oct', 'cases_nov',
       'deaths_nov']]
export_csv.to_csv("covid_19_activity.csv", index=False)
export_csv.head()

,STNAME,CTYNAME,COUNTYFP,STATE,POPESTIMATE2019,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS,cases_jan,deaths_jan,cases_feb,deaths_feb,cases_mar,deaths_mar,cases_apr,deaths_apr,cases_may,deaths_may,cases_jun,deaths_jun,cases_jul,deaths_jul,cases_aug,deaths_aug,cases_sep,deaths_sep,cases_oct,deaths_oct,cases_nov,deaths_nov
0,Alabama,Autauga County,1001,1,55869,0.053,0.074,0.134,0.295,0.444,0.0,0.0,0.0,0.0,0.0,0.0,36.0,4.0,178.0,0.0,316.0,8.0,478.0,9.0,427.0,2.0,349.0,4.0,368.0,4.0,621.0,11.0
1,Alabama,Baldwin County,1003,1,223234,0.083,0.059,0.098,0.323,0.436,0.0,0.0,0.0,0.0,0.0,0.0,155.0,3.0,116.0,6.0,390.0,1.0,2421.0,12.0,1444.0,20.0,1095.0,10.0,1300.0,19.0,1950.0,27.0
2,Alabama,Barbour County,1005,1,24686,0.067,0.121,0.120,0.201,0.491,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,128.0,0.0,158.0,0.0,273.0,4.0,159.0,2.0,139.0,0.0,164.0,2.0,118.0,2.0
3,Alabama,Bibb County,1007,1,22394,0.020,0.034,0.096,0.278,0.572,0.0,0.0,0.0,0.0,0.0,0.0,39.0,0.0,33.0,1.0,95.0,0.0,193.0,1.0,191.0,6.0,110.0,3.0,209.0,4.0,323.0,3.0
4,Alabama,Blount County,1009,1,57826,0.053,0.114,0.180,0.194,0.459,0.0,0.0,0.0,0.0,0.0,0.0,32.0,0.0,25.0,1.0,146.0,0.0,559.0,2.0,520.0,8.0,342.0,4.0,445.0,10.0,923.0,15.0
